### В этом ноутбуке готовятся данные по паркетам


In [19]:
import os
import pandas as pd
import numpy as np

import time
import gc

In [5]:
files = ['clickstream/' + file for file in sorted(os.listdir('clickstream'))]

In [6]:
pred_ts = pd.read_csv('alfabattle2_prediction_session_timestamp.csv')
pred_ts['client'] = pred_ts['client_pin']
pred_ts.drop(columns='client_pin', inplace=True)
pred_ts['session_id'] = np.arange(len(pred_ts))
pred_ts['timestamp'] = pd.to_datetime(pred_ts['timestamp'])

In [7]:

event_cat_cols = ['Accounts Cards', 'Advice', 'All Payments', 'Bank_Offer',
       'Bottom Navigation', 'Card To Card Transfer', 'Chat',
       'ClientPhoneTransfer', 'Credit Info', 'Credit Recharge', 'Investments',
       'Main Screen', 'Operations', 'Phone Bank Picker', 'Push', 'Recharge', 'ResultScreen',
       'SingleStatement', 'Widget Dashboard']

app_id_cols = ['mobile']

event_type_cols = ['pv', 'se', 'sv']

connection_cols = ['mobile', 'wifi']

In [8]:
# Собираем опорные таблицы

def collect_pivot_tables(data):
    '''
    Собираем опорные таблицы по колонкам: event_category, application_id, event_type, net_connection_type
    '''
    
    ec_pivot = pd.pivot_table(data, columns='event_category', 
                      index=['session_id'], 
                      values='timestamp', aggfunc='count')[event_cat_cols]
    
    #add_last(data, ec_pivot)
    
    app_pivot = pd.pivot_table(data, columns='application_id', 
                      index=['session_id'], 
                      values='timestamp', aggfunc='count')[app_id_cols]
    
    event_type_pivot = pd.pivot_table(data, columns='event_type', 
                          index=['session_id'], 
                          values='timestamp', aggfunc='count')[event_type_cols]
    
    connection_pivot = pd.pivot_table(data, columns='net_connection_type', 
                          index=['session_id'], 
                          values='timestamp', aggfunc='count')[connection_cols]
    
    pivots = pd.concat([ec_pivot, app_pivot, event_type_pivot, connection_pivot], axis=1).reset_index()
    pivots['session_id'] = pivots['index']
    
    return pivots.drop(columns='index').fillna(0)

In [9]:
# Собираем статистики по сессиям

def collect_sessions_stats(data):
    '''
    Собираем статистики по сессиям: продолжительность сессии, количество действий в сессии 
    и кумулятивные средние от этих средних
    '''
    
    stats = data.groupby(['session_id', 'client'])['timestamp'].agg(['first', 'last', 'count'])
    stats['duration'] = (stats['last'] - stats['first']).dt.total_seconds()
    
    stats.sort_values(['client', 'last'], inplace=True)
    
    stats['stats_num'] = stats.groupby('client')['first'].transform(lambda x: np.arange(len(x)))
    stats['cum_duration'] = stats.groupby('client')['duration'].transform(lambda x: x.expanding().mean())                                                                     

    return stats.drop(columns=['first', 'last'])

In [10]:
cols_to_drop = ['session_id_x']

In [16]:
if not os.path.exists('Datasets'):
    os.mkdir('Datasets')

In [ ]:
# Готовим данные по сессиям

start_time = time.time()

for_test = []

for file in files:
    data = pd.read_parquet(file)
    data = pd.concat([data, pred_ts[pred_ts.client.isin(data['client'].unique())]])
    
    data.sort_values(['client', 'timestamp'], inplace=True)
    data['diff_'] = data.groupby('client')['timestamp'].last().diff().dt.total_seconds() / 60 / 60
    
    temp = data.groupby(['client', 'session_id'])[['timestamp', 'timezone']].last().reset_index()
    temp.sort_values(['client', 'timestamp'], inplace=True)
    temp['prev_session1'] = temp.groupby('client')['session_id'].shift()
    
    temp1 = collect_pivot_tables(data)
    
    session_stats = collect_sessions_stats(data)
    temp1 = pd.merge(session_stats, temp1, how='left', on='session_id')
    
    del session_stats, data
    
    temp = pd.merge(temp1, temp, how='left', left_on='session_id', right_on='prev_session1')
    temp.drop(columns=cols_to_drop).to_csv('Datasets/' + file.split('/')[1], index=False)
    
    del temp, temp1
    gc.collect() 
        
    print(file)
    
finish_time = time.time()
print("\n")
print(f'processing takes {(finish_time - start_time) / 60} minutes')

clickstream/part-00000.parquet
clickstream/part-00001.parquet
clickstream/part-00002.parquet
clickstream/part-00003.parquet
clickstream/part-00004.parquet
